# Домашняя работа №5

# Задание

#### Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией
    Подбор параметров:
    - Loss
    - no_components
    - regularization

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [3]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## 1. Filter items

In [5]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


C:\Users\Андрей Сильных\Jupyter_Notebooks\Рекомендательные системы\Урок 5 (Поиск похожих товаров и пользователей. Гибридные рекомендательные системы)\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [6]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [7]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [9]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


## Encoding features

In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def fit_predict_model(loss='bpr', no_components=40, item_alpha=0.4, user_alpha=0.1):
    # init model
    model = LightFM(no_components=no_components,
                    loss=loss, # "logistic","bpr", "warp"
                    learning_rate=0.01, 
                    item_alpha=item_alpha,
                    user_alpha=user_alpha, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    # Train
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=2, # Уменьшу, так как в один поток только работает
              num_threads=12,
              verbose=False) 
    # подготавливаемм id для юзеров и товаров в порядке пар user-item
    users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    # модель возвращает меру/скор похожести между соответствующим пользователем и товаром
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=12)
    # добавляем наш полученный скор в трейн датафрейм
    data_train_filtered['score'] = predictions
    # создаем предикт датафрейм в формате списка това
    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999]
    predict_result = predict_result.drop_duplicates().sort_values(by=['user_id','score'], ascending=False)
    predict_result = predict_result.groupby('user_id')['item_id'].unique().reset_index()
    # объединяем предикт и тест датасет для подсчета precision
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
    return precision

In [18]:
%%time
_result = {}
indicators = ['loss', 'no_components', 'item_alpha', 'user_alpha', 'precision']
for el in indicators:
    _result[el] = []
for loss in ["bpr", "warp"]: # "logistic" - не заработал
    for no_components in tqdm(np.arange(40, 100, 20)):
        for item_alpha in tqdm(np.arange(0., 0.4, 0.2)):
            for user_alpha in tqdm(np.arange(0., 0.4, 0.2)):
                precision = fit_predict_model(loss, no_components, item_alpha, user_alpha)
                for el in indicators:
                    _result[el].append(eval(el))                    

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████████████████████████                                          | 1/2 [00:50<00:50, 50.04s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:41<00:00, 50.67s/it]

 50%|█████████████████████████████████████████▌                                         | 1/2 [01:41<01:41, 101.34s/it]

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████████████████████████                                          | 1/2 [00:48<00:48, 48.28s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:39<00:00, 49.66s/it]

  0%|                           

In [24]:
df = pd.DataFrame(_result).sort_values('precision', ascending=False)
df

,loss,no_components,item_alpha,user_alpha,precision
8,bpr,80,0.0,0.0,0.157668
4,bpr,60,0.0,0.0,0.155895
0,bpr,40,0.0,0.0,0.153826
5,bpr,60,0.0,0.2,0.153530
9,bpr,80,0.0,0.2,0.153235
1,bpr,40,0.0,0.2,0.145649
14,warp,40,0.2,0.0,0.143481
3,bpr,40,0.2,0.2,0.143284
7,bpr,60,0.2,0.2,0.142890
11,bpr,80,0.2,0.2,0.142299
